In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/184702-tu-ml-ws-23-breast-cancer/breast-cancer-diagnostic.shuf.tes.csv
/kaggle/input/184702-tu-ml-ws-23-breast-cancer/breast-cancer-diagnostic.shuf.lrn.csv
/kaggle/input/184702-tu-ml-ws-23-breast-cancer/breast-cancer-diagnostic.shuf.sol.ex.csv


# Load

In [2]:
X_trains = {}
X_tests = {}
X_trains['raw'] = pd.read_csv('/kaggle/input/184702-tu-ml-ws-23-breast-cancer/breast-cancer-diagnostic.shuf.lrn.csv')
X_tests['raw'] = pd.read_csv('/kaggle/input/184702-tu-ml-ws-23-breast-cancer/breast-cancer-diagnostic.shuf.tes.csv')
tes = X_tests['raw'].copy()

In [3]:
for df in [X_trains['raw'], X_tests['raw']]:
    df.rename(columns=lambda x: x.strip(), inplace=True)
    df.drop(columns='ID', inplace=True)
# Replace 'True' and 'False' in the 'class' column with 1 and 0
X_trains['raw']['class'] = X_trains['raw']['class'].replace({True: 1, False: 0})


In [4]:
def plot_histograms(dataframe):
    # Get numeric columns
    numeric_columns = dataframe.select_dtypes(include=np.number).columns.tolist()
    
    # Calculate number of rows needed for plotting
    num_plots = len(numeric_columns)
    num_plots_per_row = 4
    num_rows = (num_plots - 1) // num_plots_per_row + 1
    
    # Create subplots with the specified figure size
    fig, axes = plt.subplots(num_rows, num_plots_per_row, figsize=(20, 15))
    
    # Flatten the axes array to handle different cases
    axes = axes.flatten()
    
    # Iterate through each numeric column
    for i, column in enumerate(numeric_columns):
        # Plot histogram
        dataframe[column].plot(kind='hist', ax=axes[i], title=column)
        axes[i].set_xlabel("")  # Remove x-axis label for better layout
    
    # Hide extra subplots
    for j in range(i+1, num_plots_per_row*num_rows):
        axes[j].axis('off')
    
    plt.tight_layout()
    plt.show()

# plot_histograms(X_trains['raw'])


# Pre-process

## Log transformation

In [5]:
X_trains_log = X_trains['raw'].copy()

numeric_columns_except_class = [col for col in X_trains_log.columns if col != 'class' and pd.api.types.is_numeric_dtype(X_trains_log[col])]
X_trains_log[numeric_columns_except_class] = X_trains_log[numeric_columns_except_class].apply(lambda x: np.log1p(x))

In [6]:
def plot_histograms_transformed(dataframe):
    # Get numeric columns excluding 'class'
    numeric_columns_except_class = [col for col in dataframe.columns if col != 'class' and pd.api.types.is_numeric_dtype(dataframe[col])]
    
    # Calculate number of rows needed for plotting
    num_plots = len(numeric_columns_except_class)
    num_plots_per_row = 4
    num_rows = (num_plots - 1) // num_plots_per_row + 1
    
    # Create subplots with the specified figure size
    fig, axes = plt.subplots(num_rows, num_plots_per_row, figsize=(20, 15))
    
    # Flatten the axes array to handle different cases
    axes = axes.flatten()
    
    # Iterate through each numeric column except 'class'
    for i, column in enumerate(numeric_columns_except_class):
        # Plot histogram of the transformed data
        dataframe[column].plot(kind='hist', ax=axes[i], title=f'{column} (Transformed)')
        axes[i].set_xlabel("")  # Remove x-axis label for better layout
    
    # Hide extra subplots
    for j in range(i+1, num_plots_per_row*num_rows):
        axes[j].axis('off')
    
    plt.tight_layout()
    plt.show()

# plot_histograms_transformed(X_trains_log)


## Z-score standardisation

In [7]:
from sklearn.preprocessing import StandardScaler

X_trains_z_score = X_trains_log.copy()

numeric_columns_except_class = [col for col in X_trains_z_score.columns if col != 'class' and pd.api.types.is_numeric_dtype(X_trains_z_score[col])]
scaler = StandardScaler()
X_trains_z_score[numeric_columns_except_class] = scaler.fit_transform(X_trains_z_score[numeric_columns_except_class])


In [8]:
def plot_histograms_transformed(dataframe):
    numeric_columns_except_class = [col for col in dataframe.columns if col != 'class' and pd.api.types.is_numeric_dtype(dataframe[col])]
    
    num_plots = len(numeric_columns_except_class)
    num_plots_per_row = 4
    num_rows = (num_plots - 1) // num_plots_per_row + 1
    
    fig, axes = plt.subplots(num_rows, num_plots_per_row, figsize=(20, 15))
    
    axes = axes.flatten()
    
    for i, column in enumerate(numeric_columns_except_class):
        dataframe[column].plot(kind='hist', ax=axes[i], title=f'{column} (Transformed)')
        axes[i].set_xlabel("")  

    for j in range(i+1, num_plots_per_row*num_rows):
        axes[j].axis('off')
    
    plt.tight_layout()
    plt.show()

# plot_histograms_transformed(X_trains_z_score)


## Min-max scaling

In [9]:
from sklearn.preprocessing import MinMaxScaler

X_trains_minmax = X_trains_log.copy()

numeric_columns_except_class = [col for col in X_trains_minmax.columns if col != 'class' and pd.api.types.is_numeric_dtype(X_trains_minmax[col])]
scaler = MinMaxScaler()
X_trains_minmax[numeric_columns_except_class] = scaler.fit_transform(X_trains_minmax[numeric_columns_except_class])


In [10]:
def plot_histograms_transformed(dataframe):
    numeric_columns_except_class = [col for col in dataframe.columns if col != 'class' and pd.api.types.is_numeric_dtype(dataframe[col])]

    num_plots = len(numeric_columns_except_class)
    num_plots_per_row = 4
    num_rows = (num_plots - 1) // num_plots_per_row + 1

    fig, axes = plt.subplots(num_rows, num_plots_per_row, figsize=(20, 15))

    axes = axes.flatten()
    
    for i, column in enumerate(numeric_columns_except_class):
        dataframe[column].plot(kind='hist', ax=axes[i], title=f'{column} (Transformed)')
        axes[i].set_xlabel("") 
    
    for j in range(i+1, num_plots_per_row*num_rows):
        axes[j].axis('off')
    
    plt.tight_layout()
    plt.show()

# plot_histograms_transformed(X_trains_minmax)


# Train

In [11]:
# Splitting the log-transformed data into train and test sets
X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(X_trains_log.drop(columns=['class']), X_trains_log['class'], test_size=0.2, random_state=42)

## Classifier 1 - k-nearest neighbors

In [12]:
# Creating a new KNN Classifier
knn_log = KNeighborsClassifier()

# Training the KNN Classifier on the log-transformed training data
knn_log.fit(X_train_log, y_train_log)

# Predicting the 'class' variable on the log-transformed test set
y_pred_knn_log = knn_log.predict(X_test_log)

# Evaluating the accuracy of the KNN model using log-transformed data
accuracy_knn_log = accuracy_score(y_test_log, y_pred_knn_log)
print(f"Accuracy of KNN Classifier with Log-transformed Data: {accuracy_knn_log:.2f}")

# Cross-validation for accuracy with log-transformed data using KNN
cv_accuracy_knn_log = cross_val_score(knn_log, X_train_log, y_train_log, cv=5)  # 5-fold cross-validation
print(f"Cross-validated Accuracy with KNN and Log-transformed Data: {cv_accuracy_knn_log.mean():.2f} (+/- {cv_accuracy_knn_log.std():.2f})")

# Cross-validation for F1 score using KNN and log-transformed data
cv_f1_knn_log = cross_val_score(knn_log, X_train_log, y_train_log, cv=5, scoring='f1')  # 5-fold cross-validation
print(f"Cross-validated F1 Score with KNN and Log-transformed Data: {cv_f1_knn_log.mean():.2f} (+/- {cv_f1_knn_log.std():.2f})")


Accuracy of KNN Classifier with Log-transformed Data: 0.96
Cross-validated Accuracy with KNN and Log-transformed Data: 0.93 (+/- 0.02)
Cross-validated F1 Score with KNN and Log-transformed Data: 0.91 (+/- 0.03)


In [13]:
# Splitting the Z-score-transformed data into train and test sets
X_train_z_score, X_test_z_score, y_train_z_score, y_test_z_score = train_test_split(
    X_trains_z_score.drop(columns=['class']), X_trains_z_score['class'], test_size=0.2, random_state=42
)

# Creating a new KNN Classifier
knn_z_score = KNeighborsClassifier()

# Training the KNN Classifier on the Z-score-transformed training data
knn_z_score.fit(X_train_z_score, y_train_z_score)

# Predicting the 'class' variable on the Z-score-transformed test set
y_pred_z_score = knn_z_score.predict(X_test_z_score)

# Evaluating the accuracy of the model using Z-score-transformed data
accuracy_z_score = accuracy_score(y_test_z_score, y_pred_z_score)
print(f"Accuracy of KNN Classifier with Z-score-transformed Data: {accuracy_z_score:.2f}")

# Cross-validation for accuracy with Z-score-transformed data
cv_accuracy_z_score = cross_val_score(knn_z_score, X_train_z_score, y_train_z_score, cv=5)  # 5-fold cross-validation
print(f"Cross-validated Accuracy with Z-score-transformed Data: {cv_accuracy_z_score.mean():.2f} (+/- {cv_accuracy_z_score.std():.2f})")

# Cross-validation for F1 score using knn_z_score
cv_f1_z_score = cross_val_score(knn_z_score, X_train_z_score, y_train_z_score, cv=5, scoring='f1')  # 5-fold cross-validation
print(f"Cross-validated F1 Score with Z-score-transformed Data: {cv_f1_z_score.mean():.2f} (+/- {cv_f1_z_score.std():.2f})")


Accuracy of KNN Classifier with Z-score-transformed Data: 0.95
Cross-validated Accuracy with Z-score-transformed Data: 0.96 (+/- 0.03)
Cross-validated F1 Score with Z-score-transformed Data: 0.95 (+/- 0.03)


In [14]:
# Splitting the MinMax-scaled data into train and test sets
X_train_minmax, X_test_minmax, y_train_minmax, y_test_minmax = train_test_split(
    X_trains_minmax.drop(columns=['class']), X_trains_minmax['class'], test_size=0.2, random_state=42
)

# Creating a new KNN Classifier
knn_minmax = KNeighborsClassifier()

# Training the KNN Classifier on the MinMax-scaled training data
knn_minmax.fit(X_train_minmax, y_train_minmax)

# Predicting the 'class' variable on the MinMax-scaled test set
y_pred_minmax = knn_minmax.predict(X_test_minmax)

# Evaluating the accuracy of the model using MinMax-scaled data
accuracy_minmax = accuracy_score(y_test_minmax, y_pred_minmax)
print(f"Accuracy of KNN Classifier with MinMax-scaled Data: {accuracy_minmax:.2f}")

# Cross-validation for accuracy with MinMax-scaled data
cv_accuracy_minmax = cross_val_score(knn_minmax, X_train_minmax, y_train_minmax, cv=5)  # 5-fold cross-validation
print(f"Cross-validated Accuracy with MinMax-scaled Data: {cv_accuracy_minmax.mean():.2f} (+/- {cv_accuracy_minmax.std():.2f})")

# Cross-validation for F1 score using knn_minmax
cv_f1_minmax = cross_val_score(knn_minmax, X_train_minmax, y_train_minmax, cv=5, scoring='f1')  # 5-fold cross-validation
print(f"Cross-validated F1 Score with MinMax-scaled Data: {cv_f1_minmax.mean():.2f} (+/- {cv_f1_minmax.std():.2f})")


Accuracy of KNN Classifier with MinMax-scaled Data: 0.98
Cross-validated Accuracy with MinMax-scaled Data: 0.95 (+/- 0.03)
Cross-validated F1 Score with MinMax-scaled Data: 0.94 (+/- 0.03)


In [15]:
# Export submission
scaler = StandardScaler()
tes_z = tes.copy()
tes_z.rename(columns=lambda x: x.strip(), inplace=True)
tes_z[numeric_columns_except_class] = scaler.fit_transform(tes_z[numeric_columns_except_class])
pred = knn_z_score.predict(tes_z.drop('ID', axis = 'columns'))
sub = pd.DataFrame({'ID': tes['ID'], 'class': pred})
sub['class'] = sub['class'].apply(lambda x: bool(x))
sub.to_csv('submission.csv', index=False)
sub

,ID,class
0,884437,False
1,901011,False
2,865432,False
3,894329,False
4,84348301,True
...,...,...
279,863030,True
280,872113,False
281,915276,False
282,867739,True


## Classifier 2 - Decision tree

In [16]:
# Creating a new Decision Tree Classifier
clf_log = DecisionTreeClassifier(random_state=42)

# Training the Decision Tree Classifier on the log-transformed training data
clf_log.fit(X_train_log, y_train_log)

# Predicting the 'class' variable on the log-transformed test set
y_pred_log = clf_log.predict(X_test_log)

# Evaluating the accuracy of the model using log-transformed data
accuracy_log = accuracy_score(y_test_log, y_pred_log)
print(f"Accuracy of Decision Tree Classifier with Log-transformed Data: {accuracy_log:.2f}")

# Cross-validation for accuracy with log-transformed data
cv_accuracy_log = cross_val_score(clf_log, X_train_log, y_train_log, cv=5)  # 5-fold cross-validation
print(f"Cross-validated Accuracy with Log-transformed Data: {cv_accuracy_log.mean():.2f} (+/- {cv_accuracy_log.std():.2f})")

# Cross-validation for F1 score using clf_log
cv_f1_log = cross_val_score(clf_log, X_train_log, y_train_log, cv=5, scoring='f1')  # 5-fold cross-validation
print(f"Cross-validated F1 Score with Log-transformed Data: {cv_f1_log.mean():.2f} (+/- {cv_f1_log.std():.2f})")


Accuracy of Decision Tree Classifier with Log-transformed Data: 0.93
Cross-validated Accuracy with Log-transformed Data: 0.95 (+/- 0.03)
Cross-validated F1 Score with Log-transformed Data: 0.93 (+/- 0.04)


In [17]:
X_train_z_score, X_test_z_score, y_train_z_score, y_test_z_score = train_test_split(X_trains_z_score.drop(columns=['class']), X_trains_z_score['class'], test_size=0.2, random_state=42)

clf_z_score = DecisionTreeClassifier(random_state=42)

clf_z_score.fit(X_train_z_score, y_train_z_score)

y_pred_z_score = clf_z_score.predict(X_test_z_score)

accuracy_z_score = accuracy_score(y_test_z_score, y_pred_z_score)
print(f"Accuracy of Decision Tree Classifier with Z-score-transformed Data: {accuracy_z_score:.2f}")

cv_accuracy_z_score = cross_val_score(clf_z_score, X_train_z_score, y_train_z_score, cv=5)  # 5-fold cross-validation
print(f"Cross-validated Accuracy with Z-score-transformed Data: {cv_accuracy_z_score.mean():.2f} (+/- {cv_accuracy_z_score.std():.2f})")

cv_f1_z_score = cross_val_score(clf_z_score, X_train_z_score, y_train_z_score, cv=5, scoring='f1')  # 5-fold cross-validation
print(f"Cross-validated F1 Score with Z-score-transformed Data: {cv_f1_z_score.mean():.2f} (+/- {cv_f1_z_score.std():.2f})")


Accuracy of Decision Tree Classifier with Z-score-transformed Data: 0.93
Cross-validated Accuracy with Z-score-transformed Data: 0.95 (+/- 0.03)
Cross-validated F1 Score with Z-score-transformed Data: 0.93 (+/- 0.04)


In [18]:
X_train_minmax, X_test_minmax, y_train_minmax, y_test_minmax = train_test_split(X_trains_minmax.drop(columns=['class']), X_trains_minmax['class'], test_size=0.2, random_state=42)

clf_minmax = DecisionTreeClassifier(random_state=42)

clf_minmax.fit(X_train_minmax, y_train_minmax)

y_pred_minmax = clf_minmax.predict(X_test_minmax)

accuracy_minmax = accuracy_score(y_test_minmax, y_pred_minmax)
print(f"Accuracy of Decision Tree Classifier with MinMax-scaled Data: {accuracy_minmax:.2f}")

cv_accuracy_minmax = cross_val_score(clf_minmax, X_train_minmax, y_train_minmax, cv=5)  # 5-fold cross-validation
print(f"Cross-validated Accuracy with MinMax-scaled Data: {cv_accuracy_minmax.mean():.2f} (+/- {cv_accuracy_minmax.std():.2f})")

cv_f1_minmax = cross_val_score(clf_minmax, X_train_minmax, y_train_minmax, cv=5, scoring='f1')  # 5-fold cross-validation
print(f"Cross-validated F1 Score with MinMax-scaled Data: {cv_f1_minmax.mean():.2f} (+/- {cv_f1_minmax.std():.2f})")


Accuracy of Decision Tree Classifier with MinMax-scaled Data: 0.93
Cross-validated Accuracy with MinMax-scaled Data: 0.95 (+/- 0.03)
Cross-validated F1 Score with MinMax-scaled Data: 0.93 (+/- 0.04)


## Classifier 3 - Logistic Regression

In [19]:
# Splitting the log-transformed data into train and test sets
X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(
    X_trains_log.drop(columns=['class']), X_trains_log['class'], test_size=0.2, random_state=42
)

# Creating a new Logistic Regression Classifier
lr_log = LogisticRegression(random_state=42)

# Training the Logistic Regression Classifier on the log-transformed training data
lr_log.fit(X_train_log, y_train_log)

# Predicting the 'class' variable on the log-transformed test set
y_pred_log = lr_log.predict(X_test_log)

# Evaluating the accuracy of the model using log-transformed data
accuracy_log = accuracy_score(y_test_log, y_pred_log)
print(f"Accuracy of Logistic Regression Classifier with Log-transformed Data: {accuracy_log:.2f}")

# Cross-validation for accuracy with log-transformed data
cv_accuracy_log = cross_val_score(lr_log, X_train_log, y_train_log, cv=5)  # 5-fold cross-validation
print(f"Cross-validated Accuracy with Log-transformed Data: {cv_accuracy_log.mean():.2f} (+/- {cv_accuracy_log.std():.2f})")

# Cross-validation for F1 score using lr_log
cv_f1_log = cross_val_score(lr_log, X_train_log, y_train_log, cv=5, scoring='f1')  # 5-fold cross-validation
print(f"Cross-validated F1 Score with Log-transformed Data: {cv_f1_log.mean():.2f} (+/- {cv_f1_log.std():.2f})")

Accuracy of Logistic Regression Classifier with Log-transformed Data: 0.96
Cross-validated Accuracy with Log-transformed Data: 0.93 (+/- 0.02)
Cross-validated F1 Score with Log-transformed Data: 0.91 (+/- 0.03)


In [20]:
# Splitting the Z-score-transformed data into train and test sets
X_train_z_score, X_test_z_score, y_train_z_score, y_test_z_score = train_test_split(
    X_trains_z_score.drop(columns=['class']), X_trains_z_score['class'], test_size=0.2, random_state=42
)

# Creating a new Logistic Regression Classifier
lr_z_score = LogisticRegression(random_state=42)

# Training the Logistic Regression Classifier on the Z-score-transformed training data
lr_z_score.fit(X_train_z_score, y_train_z_score)

# Predicting the 'class' variable on the Z-score-transformed test set
y_pred_z_score = lr_z_score.predict(X_test_z_score)

# Evaluating the accuracy of the model using Z-score-transformed data
accuracy_z_score = accuracy_score(y_test_z_score, y_pred_z_score)
print(f"Accuracy of Logistic Regression Classifier with Z-score-transformed Data: {accuracy_z_score:.2f}")

# Cross-validation for accuracy with Z-score-transformed data
cv_accuracy_z_score = cross_val_score(lr_z_score, X_train_z_score, y_train_z_score, cv=5)  # 5-fold cross-validation
print(f"Cross-validated Accuracy with Z-score-transformed Data: {cv_accuracy_z_score.mean():.2f} (+/- {cv_accuracy_z_score.std():.2f})")

# Cross-validation for F1 score using lr_z_score
cv_f1_z_score = cross_val_score(lr_z_score, X_train_z_score, y_train_z_score, cv=5, scoring='f1')  # 5-fold cross-validation
print(f"Cross-validated F1 Score with Z-score-transformed Data: {cv_f1_z_score.mean():.2f} (+/- {cv_f1_z_score.std():.2f})")

Accuracy of Logistic Regression Classifier with Z-score-transformed Data: 0.96
Cross-validated Accuracy with Z-score-transformed Data: 0.99 (+/- 0.02)
Cross-validated F1 Score with Z-score-transformed Data: 0.98 (+/- 0.02)


In [21]:
# Splitting the MinMax-scaled data into train and test sets
X_train_minmax, X_test_minmax, y_train_minmax, y_test_minmax = train_test_split(
    X_trains_minmax.drop(columns=['class']), X_trains_minmax['class'], test_size=0.2, random_state=42
)

# Creating a new Logistic Regression Classifier
lr_minmax = LogisticRegression(random_state=42)

# Training the Logistic Regression Classifier on the MinMax-scaled training data
lr_minmax.fit(X_train_minmax, y_train_minmax)

# Predicting the 'class' variable on the MinMax-scaled test set
y_pred_minmax = lr_minmax.predict(X_test_minmax)

# Evaluating the accuracy of the model using MinMax-scaled data
accuracy_minmax = accuracy_score(y_test_minmax, y_pred_minmax)
print(f"Accuracy of Logistic Regression Classifier with MinMax-scaled Data: {accuracy_minmax:.2f}")

# Cross-validation for accuracy with MinMax-scaled data
cv_accuracy_minmax = cross_val_score(lr_minmax, X_train_minmax, y_train_minmax, cv=5)  # 5-fold cross-validation
print(f"Cross-validated Accuracy with MinMax-scaled Data: {cv_accuracy_minmax.mean():.2f} (+/- {cv_accuracy_minmax.std():.2f})")

# Cross-validation for F1 score using lr_minmax
cv_f1_minmax = cross_val_score(lr_minmax, X_train_minmax, y_train_minmax, cv=5, scoring='f1')  # 5-fold cross-validation
print(f"Cross-validated F1 Score with MinMax-scaled Data: {cv_f1_minmax.mean():.2f} (+/- {cv_f1_minmax.std():.2f})")

Accuracy of Logistic Regression Classifier with MinMax-scaled Data: 0.98
Cross-validated Accuracy with MinMax-scaled Data: 0.97 (+/- 0.03)
Cross-validated F1 Score with MinMax-scaled Data: 0.97 (+/- 0.04)
